In [8]:
# Import tdmclient Notebook environment:
import tdmclient.notebook
#from ipywidgets import interact, interactive, fixed, interact_manual
#import ipywidgets as widgets
import numpy as np
#await tdmclient.notebook.start()
#import import_ipynb
import PathPlanning
from ekf import ExtendedKalmanFilter

In [2]:
########   GLOBAL VARIABLES WILL ONLY BE DECLARED IN THE MAIN, AND GIVEN TO ANY FUNCTION THAT NEEDS IT ! ########



obstThrL = 500      # low obstacle threshold to switch state 1->0
obstThrH = 900      # high obstacle threshold to switch state 0->1
obstSpeedGain = [0.1, 0.06, 0, -0.06, -0.1]  # gain for motor speed necessary to avoid obstacle

current_target_idx = 0
current_plan = 0           # 0=A* algorithm, 1=obstacle avoidance
obst  = [0,0,0,0,0]  # measurements from left and right prox sensors
#### Need to define them directly as global vairables. Can't define them AFTER as global variables

@tdmclient.notebook.sync_var 
def obstacleAvoidance(position_x, position_y):
    """
    This function is used to detect an obstacle, not present while doing the path planning, and overcome it without colliding.
    The Thymio will continue to turn until he reaches the first target after the obstacle, from his global path planning.
    :param obst: a row vector of 5 values from the proximity sensors in front of the Thymio 
    :param kalman_position: actual postion of the robot 
    :param path: matrix composed by coordinates (x,y) of the global path planning
    :param saved_pos: index from 
    """
    global prox_horizontal, obst

    # acquisition from the proximity sensors to detect obstacles
    #obst = [prox_horizontal[0], prox_horizontal[1], prox_horizontal[2], prox_horizontal[3], prox_horizontal[4]]

    path = Global_navigation.getPath() # get all targets from Global navigation

    # store the kalman_position from path before a local obstacle is detected
    saved_pos = checkState(position_x, position_y)
    
    # store the index of the kalman_position from path before a local obstacle is detected
    for i in range(len(path)):
        if path[i] == saved_pos:
            saved_pos_idx = i
    return saved_pos_idx

# Function to evaluate if passing the obstacle clockwise or counterclockwise
def direction():
    """
    Returns the direction to take to overcome the obstacle (clockwise or counterclockwise) 
    :param obst: a row vector of 5 values from the proximity sensors in front of the Thymio 
    :return turn: 1x2 vector containing the clockwise movement in the first column and the counterclockwise movement 
                  in the second column. It stores a binary value 0 if the corresponding movement is locked and 1 if unlocked
    """

    # If the obstacle detected is on the right side of the Thymio, then take the clockwise direction,
    # otherwise take the counter-clockwise direction
    if (obst[3] + obst[4]) > (obst[0] + obst[1]):
        turn = 1 # clockwise turn
    elif (obst[3] + obst[4]) < (obst[0] + obst[1]):
        turn = 0 # counter-clockwise turn
    return turn


def checkState(position_x, position_y):
    """
    Returns the state of the Thymio which indicate whether to follow the path planning 
    from the A* algorithm or to avoid a local obstacle 
    :param kalman_position: the real time kalman_position of the Thymio  
    :return saved_pos: kalman_position of the Thymio before finding an obstacle 
    """
    global obst, current_plan, obstThrH, obstThrL

    # acquisition from the proximity sensors to detect obstacles
    obst = [prox_horizontal[0], prox_horizontal[1], prox_horizontal[2], prox_horizontal[3], prox_horizontal[4]]
        
    if current_plan == 0: 
        # switch from goal tracking to obst avoidance if obstacle detected
        if (max(obst) > obstThrH):
            current_plan = 1
            saved_pos = [position_y, position_x]                        
            return saved_pos
    elif current_plan == 1:
        # switch from obst avoidance to goal tracking if obstacle got unseen 
        if max(obst) < obstThrL:
            current_plan = 0
            saved_pos = 0
            return saved_pos
        
@tdmclient.notebook.sync_var


def motionControl(kalman_position, path): 
    global motor_right_target, motor_left_target, saved_pos_idx, obstSpeedGain

    thymio_speed = 100
    range_angle = 10
    turn_factor = 50

    ## convert path targets in mm instead of squares or convert measurements from kalman in squares
    square_size = 10 ## mm
    # path = path*square_size
    position_x, position_y, position_angle = kalman_position
    position_angle = position_angle*180/np.pi
    position_x = round(position_x/square_size)
    position_y = round(position_y/square_size)

    if current_plan == 0:
        if [position_y, position_x] != path[current_target_idx]:
            current_target_idx += 1

        targetAngle = np.arctan2((path[current_target_idx][1] - position_x)/(path[current_target_idx][0] - position_y))*180/np.pi %(2*np.pi)

        diff = position_angle - targetAngle 
        if abs(diff) < range_angle:
            diff = 0
        motor_left_target = thymio_speed + diff*turn_factor
        motor_right_target = thymio_speed - diff*turn_factor

    elif current_plan == 1:
        
        # adjustment for obstacles
        turn = ((obst[3] + obst[4]) > (obst[0] + obst[1])) ## or turn = direction(obst)
        # true if turning clockwise, false if turning counter-clockwise
        for i in range(5):
            spLeft += prox_horizontal[i] * obstSpeedGain[i] 
            spRight += prox_horizontal[i] * obstSpeedGain[4 - i] 
        # turn clockwise if proximity sensors value is below the threshold low
        if turn:
            if max(obst[3:4]) < obstThrL:
                spLeft = 200
                spRight = 100
                obstacleAvoidance(position_x, position_y)
                # continue turning until the original path is not found 
                if [position_y, position_x] == [path[saved_pos_idx][0], path[saved_pos_idx][1]]:
                    #turn = 0
                    current_plan = 0
                if [position_y, position_x] == [path[saved_pos_idx+1][0], path[saved_pos_idx+1][1]]:
                    #turn = 0
                    current_plan = 0
                if [position_y, position_x] == [path[saved_pos_idx+2][0], path[saved_pos_idx+2][1]]:
                    #turn = 0
                    current_plan = 0
        # turn counterclockwise if proximity sensors value is below the threshold low
        if not turn:
            if max(obst[0:1]) < obstThrL:
                    spLeft = 100
                    spRight = 200
                    obstacleAvoidance(position_x, position_y)
                    # continue turning until the original path is not found
                    if [position_y, position_x] == [path[saved_pos_idx][0], path[saved_pos_idx][1]]:
                        #turn = 0
                        current_plan = 0
                    if [position_y, position_x] == [path[saved_pos_idx+1][0], path[saved_pos_idx+1][1]]:
                        #turn = 0
                        current_plan = 0
                    if [position_y, position_x] == [path[saved_pos_idx+2][0], path[saved_pos_idx+2][1]]:
                        #turn = 0
                        current_plan = 0
        # motor control
        motor_left_target = spLeft
        motor_right_target = spRight


def currentPositionIndex(kalman_position, path):
    global current_target_idx

    position_x, position_y, position_angle = kalman_position
    if [position_y, position_x] == path[current_target_idx+1]:
        current_target_idx += 1

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 38)

In [1]:
ppouet = [[0, 1, 2, 3],[4, 5, 6, 7]]

print(ppouet)

print(ppouet[0][0])
print(ppouet[1][0])

obst = [1,9,3,4,5]
turn = ((obst[3] + obst[4]) > (obst[0] + obst[1]))

print(turn)


[[0, 1, 2, 3], [4, 5, 6, 7]]
0
4
False
